In [1]:
%run 0.0_Dependance.ipynb

In [2]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

for i in ['Bilirubin','Cholesterol','Albumin', 'Copper','Alk_Phos',
       'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']:
    if df[i].dtypes != "object":
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df["extreme_" + i] = df[i].apply(lambda x: 1 if (x < lower_bound or x > upper_bound) else 0)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from sklearn.preprocessing import RobustScaler

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)

scaler = RobustScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
from joblib import load, dump

df_score_best_test = load("df_score_best_test.joblib")
model = XGBClassifier()
comparaison = []
for j,i in enumerate(df_score_best_test["Best_param"]):
    best_model=XGBClassifier(**i)
    best_model.fit(X_train,y_train)
    probs = best_model.predict_proba(X_test)
    logloss = log_loss(y_test, probs)
    delta = logloss-df_score_best_test["LogLoss"].values[j]
    comparaison.append(delta)

In [4]:
comparaison

[0.0030836955622207673,
 -0.00010849649058708444,
 -0.002108076261859604,
 0.0007092781853603181,
 0.0006516496611435607]